In [ ]:
!pip install huggingsound
!pip install fasttext-langdetect
!pip install wget

In [ ]:
!pip install setuptools-rust
!pip install git+https://github.com/openai/whisper.git 

In [ ]:
!unzip /content/audio2.zip

Archive:  /content/audio2.zip
   creating: audio2/
  inflating: audio2/aidar1.mp3       
  inflating: audio2/aidar2.mp3       
  inflating: audio2/aidar2.wav       
  inflating: audio2/aidar3.mp3       
  inflating: audio2/aidar3.wav       
  inflating: audio2/aidar4.mp3       
  inflating: audio2/aidar4.wav       
  inflating: audio2/aidar5.mp3       
  inflating: audio2/aidar5.wav       


In [ ]:
import pandas as pd

In [ ]:
"""
model_1 = SpeechRecognitionModel("jonatasgrosman/wav2vec2-xls-r-1b-russian") # 59
model_2 = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-russian") # быстрее всех - 22 - 1.26 GB
model_3 = SpeechRecognitionModel("anton-l/wav2vec2-large-xlsr-53-russian") # 28 сджойненые токены
model_4 = SpeechRecognitionModel("facebook/s2t-small-mustc-en-ru-st")
model_5 = SpeechRecognitionModel("emre/wav2vec2-xls-r-300m-Russian-small") # 22 сджойненые токены
model_6 = SpeechRecognitionModel("jonatasgrosman/exp_w2v2t_ru_hubert_s818") # 28 - криво 
model_7 = SpeechRecognitionModel("voidful/wav2vec2-xlsr-multilingual-56") # 27 - криво
model_8 = SpeechRecognitionModel("voidful/wav2vec2-xlsr-multilingual-56") # 23 - вроде тоже норм - 23 - 1.26Gb
"""

import os
import time
import pandas as pd
import torch
from ftlangdetect import detect
from huggingsound import SpeechRecognitionModel
import whisper

In [ ]:
transcriptions = pd.read_csv('transcriptions.csv',';')\
  .rename(columns={'transcription':'nice_transcription'})

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
AUDIO_PATH = '/content/audio2/'

f = []
for (dirpath, dirnames, filenames) in os.walk(AUDIO_PATH):
    f.extend(filenames)

audio_paths = sorted(f)

In [ ]:
class SpeechTranscribation:

  def __init__(self, model_name: str):
    self.s2t_model = SpeechRecognitionModel(model_name)
    _, _, _, _, self.apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models', model='silero_te')

  def transform(self, audio_path: str):
    transcription = self.s2t_model.transcribe(audio_path)[0]['transcription']
    #lan = detect(text=transcription, low_memory=False)['lang']
    return self.apply_te(transcription, lan='ru')

In [ ]:
# init models
model = whisper.load_model("tiny")
#model_small = whisper.load_model("small")
s2t_model_1 = SpeechTranscribation('voidful/wav2vec2-xlsr-multilingual-56')
s2t_model_2 = SpeechTranscribation('jonatasgrosman/wav2vec2-large-xlsr-53-russian')

100%|█████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 91.8MiB/s]
INFO:huggingsound.speech_recognition.model:Loading model...


Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


In [ ]:
whisper_tiny_time = []
#whisper_small_time = []
s2t_1_time = []
s2t_2_time = []

whisper_tiny_text = []
#whisper_small_text = []
s2t_1_text = []
s2t_2_text = []

In [ ]:
for audio in audio_paths:
  cur_path = AUDIO_PATH+audio
  st = time.time()
  s2t_1_text.append(s2t_model_1.transform([cur_path]))
  s2t_1_time.append(time.time()-st)

  st = time.time()
  s2t_2_text.append(s2t_model_2.transform([cur_path]))
  s2t_2_time.append(time.time()-st)
  
  st = time.time()
  whisper_tiny_text.append(model.transcribe(cur_path)['text'])
  whisper_tiny_time.append(time.time()-st)

  pd.DataFrame([audio_paths, whisper_tiny_text, whisper_tiny_time, s2t_1_text, s2t_1_time, s2t_2_text, s2t_2_time]).T.to_csv('exp.csv')
  # st = time.time()
  # whisper_small_text.append(model_small.transcribe(cur_path)['text'])
  # whisper_small_time.append(time.time()-st)

In [ ]:
transcriptions

In [ ]:
itogi = pd.DataFrame([audio_paths, whisper_tiny_text, whisper_tiny_time, s2t_1_text, s2t_1_time, s2t_2_text, s2t_2_time]).T
itogi.columns = ['sound','whisper_tiny_text','whisper_tiny_time','s2t_1_text', 's2t_1_time','s2t_2_text', 's2t_2_time']

In [ ]:
transcriptions.head(5)

,file_name,nice_transcription
0,aidar1,NaN
1,aidar2,"Здравствуйте. Где яйца? Можно да? Так, да, дав..."
2,aidar3,Погода замечательная. Сегодня работал весь ден...
3,aidar4,"С юношества, с юношества. Турция, Казахстан."
4,aidar5,Мы пошли в ресторанчик за дворцом юстиций и по...


In [ ]:
itogi

,sound,whisper_tiny_text,whisper_tiny_time,s2t_1_text,s2t_1_time,s2t_2_text,s2t_2_time,file_name
0,aidar1.mp3,,53.490751,,111.79498,Свпожи ачка!,100.446702,aidar1
1,aidar2.mp3,capparagus,129.303777,S,185.425146,Деятсяможнакакдажоватепатьсеая.,194.627323,aidar2
2,aidar2.wav,Today I have to take a look yesocks onностью ...,165.903088,S,184.495109,Деятсяможнакакдажоватепатьсеая.,195.178078,aidar2
3,aidar3.mp3,"distracted, если вы привыкШientOFF AC, не rac...",100.711127,Повозувучатселна работоду идуцелдентикобо ясни...,174.47606,Погон замечательно сильно работал Енес дему ид...,170.507363,aidar3
4,aidar3.wav,"Я должна с existentialить можно на вражение, ...",85.115669,Повозувучатселна работоду идуцелдентикобоясни ...,182.97943,Погон замечательно сильно работал Енес дему ид...,215.41327,aidar3
5,aidar4.mp3,ضرع في جميل الثانية بشكلة ليست في مخطياء وشكل...,214.572517,"Бошкортерсокуян ымуш&ккуaн, бедгекунакакилгн б...",164.208703,Бошкартвер уян ему шегку Ян Бенгерку на каккил...,147.500366,aidar4
6,aidar4.wav,صدي Conway $13 تو قر forecast,186.491385,Бшкортерсокуян дымуш&ккуaн бедгекунакакилгн Бе...,158.075899,Бошкартвер уян ему шегку Ян Бенгерку на каккил...,152.895661,aidar4
7,aidar5.mp3,"Мы пошли в ресторанчик, с дворцом неустеться,...",23.061979,Мы пошли вресторанчик здварцом Есиц поебятали ...,45.12354,Мы пошли в ресторанчик с дворцом дестец и пооб...,44.828874,aidar5
8,aidar5.wav,"Мы пошли в ресторанчик, с дворцом юстит, и по...",22.84017,Мы пошли вресторанчик Здварцом Есиц поебятали ...,43.71262,Мы пошли в ресторанчик с дворцом дестец и пооб...,44.809227,aidar5


In [ ]:
itogi['file_name'] = itogi['sound'].apply(lambda x: x.split('.')[0])

In [ ]:
itogi = itogi.merge(transcriptions, on='file_name')

In [ ]:
with pd.ExcelWriter('itogi.xlsx') as wr:
  itogi.to_excel(wr)

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

rouge = Rouge()

In [ ]:
data = pd.read_excel('/content/itogi.xlsx', index_col=0).fillna('')
data.head(3)

,sound,whisper_tiny_text,whisper_tiny_time,s2t_1_text,s2t_1_time,s2t_2_text,s2t_2_time,file_name,nice_transcription
0,aidar1.ogg,,87.817444,,159.102569,Впоши Дчка.,157.597297,aidar1,
1,aidar2.ogg,You can see a little it is not genuine it is ...,273.765563,Teanos!,279.538548,Деяетсяможнакакдажовайтепкатсеаль.,276.839440,aidar2,"Здравствуйте. Где яйца? Можно да? Так, да, дав..."
2,aidar3.ogg,"Летаны Gravityברים, Polesans GlobalN collecti...",177.060808,Полозумучателна работоду дуцелодентикобоясни с...,264.978348,Погон замечательно сильно работала весь дему и...,260.845372,aidar3,Погода замечательная. Сегодня работал весь ден...


In [ ]:
whisper_scores = []
s2t_1_scores = []
s2t_2_scores = []

for text, nice_text in zip(data['whisper_tiny_text'].values, data['nice_transcription'].values):
  try:
    whisper_scores.append(rouge.get_scores(nice_text, text)[0]['rouge-1']['f'])
  except:
    whisper_scores.append(0)

for text, nice_text in zip(data['s2t_1_text'].values, data['nice_transcription'].values):
  try:
    s2t_1_scores.append(rouge.get_scores(nice_text, text)[0]['rouge-1']['f'])
  except:
    s2t_1_scores.append(0)

for text, nice_text in zip(data['s2t_2_text'].values, data['nice_transcription'].values):
  try:
    s2t_2_scores.append(rouge.get_scores(nice_text, text)[0]['rouge-1']['f'])
  except:
    s2t_2_scores.append(0)

In [ ]:
import numpy as np

In [ ]:
np.mean(whisper_scores)

0.4028400601872811

In [ ]:
np.mean(s2t_1_scores)

0.14144765399852083

In [ ]:
np.mean(s2t_2_scores)

0.39347421021261314

In [ ]:
data

In [ ]:
sdata = data.iloc[:8, :]

In [ ]:
sdata.head(10)

,sound,whisper_tiny_text,whisper_tiny_time,s2t_1_text,s2t_1_time,s2t_2_text,s2t_2_time,file_name,nice_transcription,nice_time,wh_v,st2_1_v,st2_2_v
0,aidar1.ogg,,87.817444,,159.102569,Впоши Дчка.,157.597297,aidar1,,68,1.291433,2.339744,2.317607
1,aidar2.ogg,You can see a little it is not genuine it is ...,273.765563,Teanos!,279.538548,Деяетсяможнакакдажовайтепкатсеаль.,276.839440,aidar2,"Здравствуйте. Где яйца? Можно да? Так, да, дав...",101,2.710550,2.767708,2.740985
2,aidar3.ogg,"Летаны Gravityברים, Polesans GlobalN collecti...",177.060808,Полозумучателна работоду дуцелодентикобоясни с...,264.978348,Погон замечательно сильно работала весь дему и...,260.845372,aidar3,Погода замечательная. Сегодня работал весь ден...,98,1.806743,2.703861,2.661687
3,aidar4.ogg,بين��ه knight ابن ago إن من إعلوم بن أنك,433.258121,Бoшкрдерsoкуaн еmuшokкуaн бедгеkуnаkаkилгнбирг...,254.264021,Бошкартверуян ем шегку Ян Бенгерку нак картилр...,247.712421,aidar4,"С юношества, с юношества. Турция, Казахстан.",95,4.560612,2.676463,2.607499
4,aidar5.ogg,"Мы пошли в ресторанчик, с дворцом неустеться,...",46.833074,Мы пошли в ресторанчик Здварцонесиц ообятоли З...,77.226630,Мы пошли вресторанчик с дворцом-дестиц и пообя...,75.333168,aidar5,Мы пошли в ресторанчик за дворцом юстиций и по...,39,1.200848,1.980170,1.931620
5,aidar6.ogg,В первые танок был использован в 1916 году в ...,48.070813,В первое танк было использован в тычче Деетодш...,126.398086,Впервые танк был использован в тысяча девятьсо...,123.209142,aidar6,Впервые танк был использован в тысяча девятьсо...,58,0.828807,2.179277,2.124296
6,aidar7.ogg,"В условиях, когда отмевте зависела на лечу де...",55.160956,В условиях огда отневтсе зависиго наличие дене...,116.893298,"В условиях, когда от Мефти зависило наличие де...",116.501082,aidar7,"В условиях, когда от нефти зависило наличие де...",55,1.002926,2.125333,2.118201
7,aidar8.ogg,В Майд-1985 года лидер и семьи в дущих стран ...,63.320163,в ма тичтедеветсотвосем десятьпятого года лиди...,170.096022,В мая тышятя девятьсот восемьдесят пятого года...,166.546719,aidar8,В мае тысяча девятьсот восемьдесят пятого года...,72,0.879447,2.362445,2.313149


In [ ]:
sdata['nice_time'] = times

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
sdata['wh_v'] = sdata['whisper_tiny_time']/sdata['nice_time']
sdata['st2_1_v'] = sdata['s2t_1_time']/sdata['nice_time']
sdata['st2_2_v'] = sdata['s2t_2_time']/sdata['nice_time']